In [15]:
import time
import logging
import numpy as np

In [16]:
from tictactoe import RandomAI,Board,Judge,check_win
from random import choice

## Rule
* Empty： - (0) , Player(you)： O (1) , Opponent：X (-1)
* Board Index：
| | | |
| - | - | - |
| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |

In [17]:
def profile(func):
    """
    －This function helps you to avoid wrong algorithm that cost you too many time
    －Change the limit as you wish！
    """
    def wrap(*args, **kwargs):
        limit = 10
        s = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - s
        if duration > limit:
            logging.warning(f"Time Limit Exceeded: {duration}")
        logging.info(f"using {duration} sec")
        return result
    return wrap

In [18]:
class Player:
    def __init__(self):
        self.name = "Player"
        self.player_result = {}
        self.opponent_result = {}    

    def get_valid_move(self, board_status):
        return np.where(board_status == 0)[0]

    
    def minimax(self, *kwarg):
        """
        Implement your own minmax algorithm here!
        """
        depth_left = kwarg[2]
        turn = kwarg[1]
        key = tuple(kwarg[0])
        
        # save result to optimize the running speed
        if turn == 1 and key in self.player_result:
            return self.player_result[key][0], self.player_result[key][1]
        elif turn == -1 and key in self.opponent_result:
            return self.opponent_result[key][0], self.opponent_result[key][1]
        
        if (check_win(kwarg[0]) != 0): # somebody win
            #Player win score = depth_left+1, AI win score = -(depth_left+1)
            return -turn * (depth_left + 1), -1 
        elif depth_left == 0: # tie
            return 0, -1 
        
        # best_score init for max = -infinity, min = infinity
        best_score = turn * -np.Infinity
        best_move = -1  # -1 for no more move
        moves = np.where(kwarg[0] == 0)[0]
        for move in moves:
            kwarg[0][move] = turn
            score = self.minimax(kwarg[0], -1*turn, depth_left - 1)[0]
            kwarg[0][move] = 0
            if (turn * score > turn * best_score):
                best_score, best_move = score, move
        
        if turn == 1:
            self.player_result[key] = [best_score, best_move]
        elif turn == -1:
            self.opponent_result[key] = [best_score, best_move]
        
        return best_score, best_move
        
    # kwarg[0] = board_status, kwarg[1] = turn, kwarg[2] = alpha, kwarg[3] = beta
    def alphabeta_move(self,*kwarg):
        """
        ["Optional"] Implement your own alphabeta algorithm here!
        """
        depth_left = len(self.get_valid_move(kwarg[0]))
        turn = kwarg[1]

        if (check_win(kwarg[0]) != 0): # somebody win
            #Player win score = depth_left+1, AI win score = -(depth_left+1)
            return -turn * (depth_left + 1), -1 
        elif depth_left == 0: # tie
            return 0, -1 
        
        alpha = kwarg[2]
        beta = kwarg[3]
        best_score = turn * -np.Infinity
        best_move = -1  # -1 for no more move

        # max
        if turn == 1:
            for move in np.where(kwarg[0] == 0)[0]:
                kwarg[0][move] = turn
                score = self.alphabeta_move(kwarg[0], -1*turn, alpha, beta)[0]
                kwarg[0][move] = 0
                if score > best_score:
                    best_score = score
                    best_move = move
                # max pruning
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            for move in np.where(kwarg[0] == 0)[0]:
                kwarg[0][move] = turn
                score = self.alphabeta_move(kwarg[0], -1*turn, alpha, beta)[0]
                kwarg[0][move] = 0
                if score < best_score:
                    best_score = score
                    best_move = move
                # min pruning
                if score <= alpha:
                    break
                beta = min(beta, score)
        
        return best_score, best_move
        
    def random_move(self,board_status)->int:
        return choice(self.get_valid_move(board_status))
    
    @profile
    def move(self, board_status)->int:
        """
        - Here we show you the result that use random move as strategy
        """
        #return self.minimax(board_status, 1, len(self.get_valid_move(board_status)))[1]
        return self.alphabeta_move(board_status, 1, -np.Infinity, np.Infinity)[1]
        #return self.random_move(board_status)

In [19]:
NUM_RUNS = [-1,1]*50

In [20]:
game = Board(Player(),RandomAI(), Judge(who_Turn=-1))
print("PLAYER：　Ｏ　AI：Ｘ   Space：-　"+"\n")
start = time.time()
for i in NUM_RUNS:
    game.judge.who_Turn = i
    game.play()
end = time.time()
print(f"Time cost --- {end - start}")

PLAYER：　Ｏ　AI：Ｘ   Space：-　

PLAYER　WIN 1:
[['O' 'X' 'O']
 ['X' 'O' 'X']
 ['X' '-' 'O']]

PLAYER　WIN 2:
[['O' '-' '-']
 ['X' 'O' 'X']
 ['-' '-' 'O']]

PLAYER　WIN 3:
[['O' '-' '-']
 ['X' 'O' '-']
 ['X' 'X' 'O']]

PLAYER　WIN 4:
[['O' 'X' 'O']
 ['-' 'O' 'X']
 ['O' '-' 'X']]

PLAYER　WIN 5:
[['-' 'O' 'X']
 ['-' 'O' 'X']
 ['X' 'O' '-']]

PLAYER　WIN 6:
[['O' 'O' 'O']
 ['-' 'X' '-']
 ['X' 'O' 'X']]

PLAYER　WIN 7:
[['O' '-' '-']
 ['X' 'O' 'X']
 ['-' 'X' 'O']]

PLAYER　WIN 8:
[['X' '-' 'O']
 ['X' 'O' '-']
 ['O' '-' '-']]

PLAYER　WIN 9:
[['-' 'X' 'X']
 ['O' 'O' 'O']
 ['-' '-' 'X']]

PLAYER　WIN 10:
[['O' 'O' 'O']
 ['O' 'X' 'X']
 ['X' '-' '-']]

PLAYER　WIN 11:
[['O' 'X' '-']
 ['X' 'O' '-']
 ['-' 'X' 'O']]

PLAYER　WIN 12:
[['X' '-' 'O']
 ['-' '-' 'O']
 ['-' 'X' 'O']]

PLAYER　WIN 13:
[['O' 'O' 'O']
 ['X' '-' 'X']
 ['O' 'X' 'X']]

PLAYER　WIN 14:
[['O' 'O' '-']
 ['X' 'O' '-']
 ['X' 'O' 'X']]

Tie 1:
[['X' 'X' 'O']
 ['O' 'O' 'X']
 ['X' 'O' 'X']]

PLAYER　WIN 15:
[['O' '-' '-']
 ['X' 'O' '-']
 ['X' '-' 'O']]

In [21]:
print(f"You Win {game.judge.n_player_win} out of 100 games")
print(f"You Lose {game.judge.n_player_lose} out of 100 games")
print(f"You Tie {game.judge.tie} out of 100 games")
del game

You Win 91 out of 100 games
You Lose 0 out of 100 games
You Tie 9 out of 100 games
